In [42]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d5167ac9-42fa-23c1-70d0-9b64c5ee207b)


In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [44]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import os
import re
import pickle
models = os.listdir(f"{proj_dir}saved_models/list_of_models/gen")
model_number = [int(re.sub("generators", "", x)) for x in models]
last_model = max(model_number)

In [46]:
column_names = df.columns
df = df.to_numpy()

In [47]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(df) 
# Transfo
df = scaler.transform(df)

In [48]:
# TODO lets put this away as a script
import tensorflow as tf

import numpy as np
from tensorflow import keras

def build_network(output_dim, n_hidden, n_neurons, learning_rate):

    """

    output_dim: what do we want this to output
    Generator output n_columns of data
    Discriminator output 1, p(data_real|data_seen)

    n_hiden: number of layers of the neural net

    n_neurons: number of neuros in the network

    learning_rate: duhhh

    This outputs a keras neural net
    
    """
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="selu"))
        model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(output_dim + 10, activation="selu"))  
    model.add(keras.layers.Dense(output_dim, activation="sigmoid"))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer)
    return model

def train_gan(
    generator, discriminator, dataset, n_epochs=100, n_noise=20000, accuracy=0.9999
):
    """
    # TODO: UPDATE ARGS
    Inputs: 

    gan, this is a keras gan object made by combining two neural nets and
    restricting the trainability of one of them.

    dataset, this takes in regular tabular data. now this is training rowwise
    however i may change this to matrix wise like a picture.

    n_epochs, numper of times the gans go though training iterationations

    iterationations, number of times in gan iterationaton loop, 
    it would be a good idea to reduct this after the warmup period

    n_noise, this is the size of fake data generated

    
    Output:

    generators_saved, this is an iterationable list of keras objects that can be used
    
    discriminators_saved, same thing, these can be used to test

    for generator, discriminator in zip(gen, desc):
        noise = tf.random.normal(shape=dims)
        generated_data = generator(noise)
        judgement = discriminator(generated_data) # probs data is real
    """
    gan = keras.models.Sequential([generator, discriminator])
  
    discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
    discriminator.trainable = False
    gan.compile(loss="binary_crossentropy", optimizer="rmsprop")
    generator, discriminator = gan.layers
    generators_saved = []
    discriminators_saved = []
    result_log = []
    for epoch in range(n_epochs):
        print(f"~~~~~~~~~~Epoch {epoch} of {n_epochs}~~~~~~~~~~~~~~")

        min_judgement = 0
        iteration = 0

        while min_judgement < accuracy:
            tf.random.set_seed(iteration*5)

            if iteration > 5000: # its likely stuck
              break
            
            
            # TODO: write to csv and append
            iteration += 1
            random_index = tf.random.uniform(shape=(n_noise,), minval=1, maxval=len(dataset), dtype=tf.int32)
            X_batch = dataset[random_index, :]
          

            noise = tf.random.normal(shape=X_batch.shape,
                                     mean=0,
                                     stddev=1) 


            generated_data = generator(noise)
            X_fake_and_real = tf.concat([generated_data, X_batch], axis=0)
            y1 = tf.concat([tf.zeros(n_noise), tf.ones(n_noise)], axis=0)
            
            # training discriminator
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # training the generator

            noise = tf.random.normal(shape=X_batch.shape,
                                     mean=0,
                                     stddev=1) 
            
            discriminator.trainable = False
            gan.train_on_batch(noise, tf.ones(n_noise))
            
            # testing quality of model

            # TODO:
            # Compute Covariance matrix
            # Order Stats of generated
            # classifier
            
            noise = tf.random.normal(shape=X_batch.shape,
                                     mean=0,
                                     stddev=1) 
            

            generated_data = generator(noise)
            gen = pd.DataFrame(scaler.inverse_transform(generated_data), columns=column_names)
            print(gen)
            judgement = discriminator(generated_data) # probs data is real

            results = [epoch, iteration, np.mean(judgement), np.min(judgement), np.max(judgement)]
            print(
                "\n epoch", results[0],
                "\n iteration", results[1],
                "\n mean", results[2],
                "\n min ", results[3],
                "\n max ", results[4]
                )
            min_judgement = results[2]
            result_log.append(results)
        
        generators_saved.append(generator)
        discriminators_saved.append(discriminator)
            
    return generators_saved, discriminators_saved, result_log


lets consider initializing a new gan with each epoch or 

In [49]:
generator = build_network(output_dim=df.shape[1], n_hidden=3, n_neurons=450, learning_rate=5e-3)
discriminator = build_network(output_dim=1, n_hidden=2, n_neurons=750, learning_rate=1e-3) # lower this

In [ ]:
generators_saved, discriminators_saved, result_log = train_gan(
    generator, discriminator, df, n_epochs=500, n_noise=5000, accuracy=0.999
)

~~~~~~~~~~Epoch 0 of 500~~~~~~~~~~~~~~
      year_pickup_datetime  quarter_pickup_datetime  month_pickup_datetime  \
0              2016.028277                 1.513866               5.162821   
1              2016.047617                 1.681746               5.005117   
2              2016.178327                 1.646785               5.010652   
3              2016.008216                 1.621133               4.727274   
4              2016.164190                 1.934823               3.835238   
...                    ...                      ...                    ...   
4995           2016.051579                 1.773295               5.138144   
4996           2016.313895                 1.739498               4.625688   
4997           2016.694303                 1.741131               3.651046   
4998           2016.507003                 1.707750               4.912693   
4999           2016.507439                 1.754341               3.788881   

      dayofweek_pickup_d

In [ ]:
df.mean(axis=0)

In [ ]:
with open(f"{proj_dir}saved_models/list_of_models/gen/generators{last_model+1}", "wb") as fp:
    pickle.dump(generators_saved, fp)

with open(f"{proj_dir}saved_models/list_of_models/disc/discriminators{last_model+1}", "wb") as fp:
    pickle.dump(discriminators_saved, fp)

with open(f"{proj_dir}logs/logGAN{last_model+1}", "wb") as fp:
    pickle.dump(result_log, fp)

results = pd.DataFrame(result_log, columns=["epoch", "iter", "mean", "min", "max"])
results.to_csv(f"{proj_dir}/data_clean/data_for_graphics/model{last_model+1}_convergence.csv")
  

In [ ]:

def generated_data_filter(gen, desc, points_to_gen, threashold, dims):
    """
    inputs
    gen, is the list of gans we wrote with the gan.ipynb

    desc, is the list of discriminators in the notebook gan.ipynb
    
    points_to_gen, number of datapoints for each model to generate

    threashold, is what is the discriminator's predicted probability of the data being real
    we need to see to keep the data. 
    with a threashold = 0.99 we will drop every datapoint that the discriminator says has a 
    less than .99 change of being real. 
    we will need to play with this.

    """
    n_col = dims[1]
    quality_data = np.empty((0, n_col), np.float32)

    for generator, discriminator in zip(gen, desc):
        noise = tf.random.normal(shape=(points_to_gen, n_col))
        generated_data = generator(noise)
        judgement = discriminator(generated_data) # probs data is real
        data_fooling_discriminator = np.compress(np.ravel(judgement) > threashold, generated_data, axis=0)

        quality_data = np.append(quality_data, data_fooling_discriminator, axis=0)
    
    for discriminator in desc:
        judgement = discriminator(quality_data)
        quality_data = np.compress(np.ravel(judgement) > threashold, quality_data, axis=0)
    return quality_data

In [ ]:
generated_dataset = generated_data_filter(generators_saved, discriminators_saved, points_to_gen=1000, threashold=0.99, dims=df.shape)

In [ ]:
generated_data = pd.DataFrame(scaler.inverse_transform(generated_dataset), columns=column_names) # revert data